In [50]:
import pandas as pd
import numpy as np
import datetime
import traceback
import time

pd.options.plotting.backend = 'plotly'
pd.set_option('display.float_format', '{:,.4f}'.format)

In [51]:
df = pd.read_csv('/data/tick_feature_udp/arbit_unwind_pi/VN30F1M.csv', parse_dates=["Date"])

In [52]:
daily_df = (df.groupby(df['Date'].dt.date).agg({
                'Symbol': "last", 
                'fund_name': "last", 
                'arbit_ratio': "sum", 
                'arbit_value': "sum",
                'unwind_ratio': "sum", 
                'unwind_value': "sum"  }  
            )[['arbit_ratio', 'unwind_ratio', 'arbit_value', 'unwind_value']]
            .reset_index()
            .assign(
                ab_un_diff=lambda row: row['arbit_value'] - row['unwind_value']
            ))

In [53]:
daily_df['Date'] = pd.to_datetime(daily_df['Date'])

In [54]:
daily_df.set_index('Date').ab_un_diff.plot()

In [55]:
fund_report = pd.read_csv('/home/duc/Desktop/feature_engineer/evaluation/dc/dc_report.csv', parse_dates=['Date'])

In [56]:
df_compare = daily_df[['Date', 'ab_un_diff']].merge(fund_report[['Date','primary_flow']], how='left', on='Date')
df_compare = df_compare.set_index(['Date'])

In [57]:
df_compare['2024-09-05':'2024-09-05'].rename(columns={
    'ab_un_diff': 'pi_detect',
    'primary_flow': 'dc_report'
})

,pi_detect,dc_report
Date,,
2024-09-05,"-62,532,810,000.0000","-75,872,775,561.0000"


In [58]:
df_compare = df_compare.fillna(0)
fig = df_compare.plot()
fig.data[0].name = 'Pi Detect'
fig.data[1].name = 'Report from DC website'
fig.update_layout(
    title='Daily Arbit/Unwind Value'
)
fig.show()